# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-e6a5ee3f6b-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Sherlock Holmes explorative search") 


Consider the following exploratory scenario:


> Investigate the canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes) and which are the films and tv series related to the canon. We are also interested in the Emmy Awards and hiw many awards each film won. 


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wd:Q35610` | Arthur Conan Doyle        | node |



Also consider

```
wd:Q35610 ?p ?obj .
```

is the BGP to retrieve all **properties of Arthur Conan Doyle**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes)

2. Identify the BGP for film

3. Identify the BGP for television series

4. Return the year of publication (take the first one if there are more publication dates) for each film and television series related to the canon of Sherlock Holmes (the result set must be a list of triples film/tv series IRI, label and year).

5. Find who acted in more than one film or tv series related to the canon of Sherlock Holmes (the result set must be a list of triples actor/actress IRI, label and #film).

6. Identify the BGP for Primetime Emmy Award

7. Return all the types of Primetime Emmy Award (the result set must be a list of couples award IRI and label).

8. For each film and television series related to the canon of Sherlock Holmes, return the number of Primetime Emmy Awards they won (return also the objects who did not win an Award) (the result set must be a list of triples film/tv series IRI, label and #award).

## Task 1
Identify the BGP for canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes)

In [15]:
# RETURN ALL DISTINCT PROPERTIES FOR ARTHUR CONAN DOYLE
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q35610 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4928'), ('name', 'Ricorso author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5714'), ('name', 'Tor.com author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9419'), ('name', 'personal library at')]
[('p', 'http://www.wikidata.org/prop/direct/P5470'), ('name', 'Scottish Poetry Library poet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http:/

In [11]:
#SAME WITH DIFFERENT LIMIT
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q35610 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4928'), ('name', 'Ricorso author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5714'), ('name', 'Tor.com author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9419'), ('name', 'personal library at')]
[('p', 'http://www.wikidata.org/prop/direct/P5470'), ('name', 'Scottish Poetry Library poet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http:/

In [5]:
#ARTHUR CONAN DOYLE'S LIST OF WORKS
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q35610 wdt:P1455 ?obj .
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q4798323'), ('name', 'Arthur Conan Doyle bibliography')]
1


In [6]:
#PROPERTIES FOR Arthur Conan Doyle bibliography
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q4798323 ?p ?obj .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P50'), ('name', 'author')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
4


In [7]:
#RETURN NODES INSTANCE OF Arthur Conan Doyle bibliography
queryString = """
SELECT DISTINCT ?s ?name
WHERE {
   ?s wdt:P31 wd:Q4798323.
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [8]:
#BACK TO Arthur Conan Doyle's PROPERTIES WITH LIMIT 150
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q35610 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4928'), ('name', 'Ricorso author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5714'), ('name', 'Tor.com author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9419'), ('name', 'personal library at')]
[('p', 'http://www.wikidata.org/prop/direct/P5470'), ('name', 'Scottish Poetry Library poet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http:/

In [9]:
#Arthur Conan Doyle PROPERTIES WITH LIMIT 200
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q35610 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4928'), ('name', 'Ricorso author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5714'), ('name', 'Tor.com author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9419'), ('name', 'personal library at')]
[('p', 'http://www.wikidata.org/prop/direct/P5470'), ('name', 'Scottish Poetry Library poet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http:/

In [10]:
#RETURN WHAT Arthur Conan Doyle IS INSTANCE OF
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q35610 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5'), ('name', 'human')]
1


In [12]:
#RETURN HIS OCCUPATIONS
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q35610 wdt:P106 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q11774202'), ('name', 'essayist')]
[('obj', 'http://www.wikidata.org/entity/Q18844224'), ('name', 'science fiction writer')]
[('obj', 'http://www.wikidata.org/entity/Q214917'), ('name', 'playwright')]
[('obj', 'http://www.wikidata.org/entity/Q28389'), ('name', 'screenwriter')]
[('obj', 'http://www.wikidata.org/entity/Q36180'), ('name', 'writer')]
[('obj', 'http://www.wikidata.org/entity/Q39631'), ('name', 'physician')]
[('obj', 'http://www.wikidata.org/entity/Q4853732'), ('name', "children's writer")]
[('obj', 'http://www.wikidata.org/entity/Q551835'), ('name', 'physician writer')]
[('obj', 'http://www.wikidata.org/entity/Q6625963'), ('name', 'novelist')]
[('obj', 'http://www.wikidata.org/entity/Q10297252'), ('name', 'crime writer')]
10


In [13]:
#RETURN MORE PROPERTIES OF Arthur Conan Doyle SEARCHING FOR 
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q35610 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 300
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4928'), ('name', 'Ricorso author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5714'), ('name', 'Tor.com author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9419'), ('name', 'personal library at')]
[('p', 'http://www.wikidata.org/prop/direct/P5470'), ('name', 'Scottish Poetry Library poet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http:/

In [14]:
#Arthur Conan Doyle DESCRIBED BY SOURCE
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q35610 wdt:P1343 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 300
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q16082057'), ('name', "The New Student's Reference Work")]
[('obj', 'http://www.wikidata.org/entity/Q17329836'), ('name', 'Larousse Encyclopedia online')]
[('obj', 'http://www.wikidata.org/entity/Q17378135'), ('name', 'Great Soviet Encyclopedia (1969–1978)')]
[('obj', 'http://www.wikidata.org/entity/Q19180675'), ('name', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('obj', 'http://www.wikidata.org/entity/Q3181656'), ('name', 'The Nuttall Encyclopædia')]
[('obj', 'http://www.wikidata.org/entity/Q4239850'), ('name', 'Concise Literary Encyclopedia')]
[('obj', 'http://www.wikidata.org/entity/Q4263804'), ('name', 'Literary Encyclopedia 1929—1939')]
[('obj', 'http://www.wikidata.org/entity/Q4532138'), ('name', 'Granat Encyclopedic Dictionary')]
[('obj', 'http://www.wikidata.org/entity/Q51955019'), ('name', 'BEIC Digital Library')]
[('obj', 'http://www.wikidata.org/entity/Q602358'), ('name', 'Brockhaus and Efron Encyclopedic Dictionar

In [15]:
#Arthur Conan Doyle NOTABLE WORK
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q35610 wdt:P800 ?obj .
   ?obj sc:name ?name.
}
LIMIT 300
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q2316684'), ('name', 'canon of Sherlock Holmes')]
[('obj', 'http://www.wikidata.org/entity/Q785255'), ('name', 'The Lost World')]
2


Final query for this task

In [2]:
# write your final query
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   wd:Q35610 wdt:P800 ?obj .
   ?obj sc:name ?name.
}
LIMIT 300
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q2316684'), ('name', 'canon of Sherlock Holmes')]
[('obj', 'http://www.wikidata.org/entity/Q785255'), ('name', 'The Lost World')]
2


## Task 2
Identify the BGP for film

In [3]:
#RETURN PROPERTIES FOR canon of Sherlock Holmes
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q2316684 ?p ?obj .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work')]
[('p', 'http://www.wikidata.org/prop/direct/P50'), ('name', 'author')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P674'), ('name', 'characters')]
[('p', 'http://www.wikidata.org/prop/direct/P6839'), ('name', 'TV Tropes identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P8895'), ('name', 'All the Tropes identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
11


In [4]:
#RETURN WHAT canon of Sherlock Holmes IS INSTANCE OF
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q2316684 wdt:P31 ?o .
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q277759'), ('name', 'book series')]
[('o', 'http://www.wikidata.org/entity/Q1631107'), ('name', 'bibliography')]
[('o', 'http://www.wikidata.org/entity/Q53815'), ('name', 'canon')]
3


In [5]:
#RETURN PROPERTIES AND OBJECT FOR canon of Sherlock Holmes
queryString = """
SELECT DISTINCT ?p ?name ?o ?oName
WHERE {
   wd:Q2316684 ?p ?o .
   ?p sc:name ?name.
   ?o sc:name ?oName
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work'), ('o', 'http://www.wikidata.org/entity/Q1764019'), ('oName', 'Sherlock Holmes')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('o', 'http://www.wikidata.org/entity/Q733582'), ('oName', 'His Last Bow')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('o', 'http://www.wikidata.org/entity/Q45192'), ('oName', 'The Hound of the Baskervilles')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q277759'), ('oName', 'book series')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('o', 'http://www.wikidata.org/entity/Q860831'), ('oName', 'The Memoirs of Sherlock Holmes')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q1631107'), ('oName', 'bibliography')]
[('p', 'http://www.wikidata.org/prop/direc

In [6]:
#RETURN WHAT Sherlock Holmes: A Game of Shadows IS INSTANCE OF
queryString = """
SELECT DISTINCT ?o ?name 
WHERE {
   wd:Q243643 wdt:P31 ?o .
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
1


Final query for this task

In [3]:
# write your final query
queryString = """
SELECT DISTINCT ?o ?name 
WHERE {
   wd:Q243643 wdt:P31 ?o .
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
1


## Task 3
Identify the BGP for television series

In [ ]:
# write your queries

In [7]:
#RETURN WHAT FILM IS INSTANCE OF
queryString = """
SELECT DISTINCT ?o ?name 
WHERE {
   wd:Q11424 wdt:P31 ?o .
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [8]:
#RETURN PROPERTIES FOR FILM
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   wd:Q11424 ?p ?o .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('p', 'http://www.wikidata.org/prop/direct/P1256'), ('name', 'Iconclass notation')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1368'), ('name', 'LNB ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p',

In [9]:
#RETURN FILM'S SUPERCLASS
queryString = """
SELECT DISTINCT ?o ?name 
WHERE {
   wd:Q11424 wdt:P279 ?o .
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q20937557'), ('name', 'series')]
[('o', 'http://www.wikidata.org/entity/Q4502142'), ('name', 'visual artwork')]
[('o', 'http://www.wikidata.org/entity/Q2431196'), ('name', 'audiovisual work')]
[('o', 'http://www.wikidata.org/entity/Q10301427'), ('name', 'moving image')]
4


In [10]:
#RETURN SUBCLASSES OF SERIES
queryString = """
SELECT DISTINCT ?s ?name 
WHERE {
   ?s wdt:P279 wd:Q20937557 .
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q332225'), ('name', 'endgame')]
[('s', 'http://www.wikidata.org/entity/Q98391050'), ('name', 'nuclear tests series')]
[('s', 'http://www.wikidata.org/entity/Q31836768'), ('name', 'model series')]
[('s', 'http://www.wikidata.org/entity/Q133250'), ('name', 'sequence')]
[('s', 'http://www.wikidata.org/entity/Q3249551'), ('name', 'process')]
[('s', 'http://www.wikidata.org/entity/Q11471'), ('name', 'time')]
[('s', 'http://www.wikidata.org/entity/Q52905625'), ('name', 'numismatic serie')]
[('s', 'http://www.wikidata.org/entity/Q12538685'), ('name', 'roadshow')]
[('s', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
[('s', 'http://www.wikidata.org/entity/Q15142889'), ('name', 'weapon family')]
[('s', 'http://www.wikidata.org/entity/Q1914636'), ('name', 'activity')]
[('s', 'http://www.wikidata.org/entity/Q1326471'), ('name', 'chord progression')]
[('s', 'http://www.wikidata.org/entity/Q357384'), ('name', 'cascade')]
[('s', 'http://www

In [11]:
#RETURN SUBCLASSES OF VISUAL ARTWORK
queryString = """
SELECT DISTINCT ?s ?name 
WHERE {
   ?s wdt:P279 wd:Q4502142 .
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q93184'), ('name', 'drawing')]
[('s', 'http://www.wikidata.org/entity/Q163047'), ('name', 'artificial flower')]
[('s', 'http://www.wikidata.org/entity/Q21281546'), ('name', 'gouache painting')]
[('s', 'http://www.wikidata.org/entity/Q29457028'), ('name', 'projection')]
[('s', 'http://www.wikidata.org/entity/Q20460'), ('name', 'crucifix')]
[('s', 'http://www.wikidata.org/entity/Q425382'), ('name', 'Hilya')]
[('s', 'http://www.wikidata.org/entity/Q178401'), ('name', 'amphora')]
[('s', 'http://www.wikidata.org/entity/Q3257185'), ('name', 'book-and-record set')]
[('s', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
[('s', 'http://www.wikidata.org/entity/Q133067'), ('name', 'mosaic')]
[('s', 'http://www.wikidata.org/entity/Q11060274'), ('name', 'print')]
[('s', 'http://www.wikidata.org/entity/Q3305213'), ('name', 'painting')]
[('s', 'http://www.wikidata.org/entity/Q860861'), ('name', 'sculpture')]
[('s', 'http://www.wikidata.org/en

In [12]:
#RETURN WHAT SHERLOK IS INSTANCE OF
queryString = """
SELECT DISTINCT ?o ?name 
WHERE {
   wd:Q192837 wdt:P31 ?o .
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q5398426'), ('name', 'television series')]
[('o', 'http://www.wikidata.org/entity/Q1259759'), ('name', 'miniseries')]
2


Final query for this task

In [3]:
# write your final query
queryString = """
SELECT DISTINCT ?o ?name 
WHERE {
   wd:Q192837 wdt:P31 ?o .
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q5398426'), ('name', 'television series')]
[('o', 'http://www.wikidata.org/entity/Q1259759'), ('name', 'miniseries')]
2


## Task 4
Return the year of publication (take the first one if there are more publication dates) for each film and television series related to the canon of Sherlock Holmes (the result set must be a list of triples film/tv series IRI, label and year).

In [ ]:
# write your queries

In [13]:
#RETURN PROPERTIES FOR Sherlock Holmes: A Game of Shadows
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   wd:Q243643 ?p ?o.
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('name', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('name', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('name', 'Metacritic ID')]
[('p', 'http://www.wikidata.org/pro

In [14]:
#RETURN PUBLICATION DATE
queryString = """
SELECT DISTINCT ?o ?name 
WHERE {
   wd:Q243643 wdt:P577 ?o.
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [15]:
#RETURN PUBLICATION DATE CORRECT
queryString = """
SELECT DISTINCT ?o ?name 
WHERE {
   wd:Q243643 wdt:P577 ?o.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', '2011-12-16T00:00:00Z')]
1


In [16]:
#RETURN PUBLICATION DATE CORRECT WITHOUT DISTINCT
queryString = """
SELECT ?o ?name 
WHERE {
   wd:Q243643 wdt:P577 ?o.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', '2011-12-16T00:00:00Z')]
1


In [18]:
#RETURN PUBLICATION DATE FOR ALL RELATED WORKS
queryString = """
SELECT DISTINCT ?relatedWork ?name ?date
WHERE {
   wd:Q2316684 wdt:P4969 ?relatedWork.
   ?relatedWork wdt:P577 ?date.
   ?relatedWork sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('relatedWork', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('date', '2013-01-10T00:00:00Z')]
[('relatedWork', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('date', '2013-01-13T00:00:00Z')]
[('relatedWork', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('date', '2013-01-29T00:00:00Z')]
[('relatedWork', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('date', '2013-05-03T00:00:00Z')]
[('relatedWork', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('date', '2014-01-03T00:00:00Z')]
[('relatedWork', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('date', '2009-12-24T00:00:00Z')]
[('relatedWork', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('date', '2010-01-07T00:00:00Z')]
[('relatedWork', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('date', '2010-01-28T00:00:00Z')]
[('relatedWork', 'http://www.wiki

In [22]:
#RETURN PUBLICATION FIRST YEAR FOR ALL RELATED WORKS
queryString = """
SELECT DISTINCT ?relatedWork ?name (year(MIN(?date)) AS ?firstYear)
WHERE {
   wd:Q2316684 wdt:P4969 ?relatedWork.
   ?relatedWork wdt:P577 ?date.
   ?relatedWork sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('relatedWork', 'http://www.wikidata.org/entity/Q1210893'), ('name', 'The Adventures of Sherlock Holmes and Dr. Watson'), ('firstYear', '1979')]
[('relatedWork', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('firstYear', '2013')]
[('relatedWork', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('firstYear', '2009')]
[('relatedWork', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows'), ('firstYear', '2011')]
4


In [23]:
#RETURN PROPERTIES FOR SHERLOCK HOLMES 
queryString = """
SELECT ?p ?name 
WHERE {
   wd:Q387306 ?p ?o.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P227')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646')]
[('p', 'http://www.wikidata.org/prop/direct/P1113')]
[('p', 'http://www.wikidata.org/prop/direct/P1267')]
[('p', 'http://www.wikidata.org/prop/direct/P136')]
[('p', 'http://www.wikidata.org/prop/direct/P136')]
[('p', 'http://www.wikidata.org/prop/direct/P144')]
[('p', 'http://www.wikidata.org/prop/direct/P1476')]
[('p', 'http://www.wikidata.org/prop/direct/P161')]
[('p', 'http://www.wikidata.org/prop/direct/P161')]
[('p', 'http://www.wikidata.org/prop/direct/P161')]
[('p', 'http://www.wikidata.org/prop/direct/P161')]
[('p', 'http://www.wikidata.org/prop/direct/P161')]
[('p', 'http://www.wikidata.org/prop/direct/P161')]
[('p', 'http://www.wikidata.org/prop/direct/P1811')]
[('p', 'http://www.wikidata.org/prop/direct/P2047')]
[('p', 'http://www.wikidata.org/prop/direct/P227')]
[('p', 'http://www.wik

In [25]:
#RETURN PROPERTIES FOR SHERLOCK HOLMES AND LABELS
queryString = """
SELECT ?p ?name 
WHERE {
   wd:Q387306 ?p ?o.
   ?p sc:name ?name
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P1811'), ('name', 'list of e

In [26]:
#RETURN FIRST PUBLICATION DATE OR START TIME CASTED TO YEAR FOR ALL RELATED WORKS
queryString = """
SELECT DISTINCT ?relatedWork ?name (year(MIN(?date)) AS ?firstYear)
WHERE {
   wd:Q2316684 wdt:P4969 ?relatedWork.
   ?relatedWork wdt:P577 | wdt:P580 ?date.
   ?relatedWork sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('relatedWork', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes'), ('firstYear', '1984')]
[('relatedWork', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes'), ('firstYear', '2013')]
[('relatedWork', 'http://www.wikidata.org/entity/Q1210893'), ('name', 'The Adventures of Sherlock Holmes and Dr. Watson'), ('firstYear', '1979')]
[('relatedWork', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('firstYear', '2012')]
[('relatedWork', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('firstYear', '2009')]
[('relatedWork', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('firstYear', '2010')]
[('relatedWork', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows'), ('firstYear', '2011')]
[('relatedWork', 'http://www.wikidata.org/entity/Q1764019'), ('name', 'Sherlock Holmes'), ('firstYear', '1939')]
8


In [27]:
#CHECK IF ALL ARE PRESENT 
queryString = """
SELECT DISTINCT ?o ?name
WHERE {
   wd:Q2316684 wdt:P4969 ?o .
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1764019'), ('name', 'Sherlock Holmes')]
[('o', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock')]
[('o', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary')]
[('o', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes')]
[('o', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes')]
[('o', 'http://www.wikidata.org/entity/Q1210893'), ('name', 'The Adventures of Sherlock Holmes and Dr. Watson')]
[('o', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes')]
[('o', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows')]
8


In [36]:
#CHECK IF THEY ARE ALL FILM OR TV SERIES
queryString = """
SELECT DISTINCT ?o ?name ?class ?className
WHERE {
   wd:Q2316684 wdt:P4969 ?o .
   ?o wdt:P31 ?class.
   ?o sc:name ?name.
   ?class sc:name ?className.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1764019'), ('name', 'Sherlock Holmes'), ('class', 'http://www.wikidata.org/entity/Q24856'), ('className', 'film series')]
[('o', 'http://www.wikidata.org/entity/Q1210893'), ('name', 'The Adventures of Sherlock Holmes and Dr. Watson'), ('class', 'http://www.wikidata.org/entity/Q24856'), ('className', 'film series')]
[('o', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('class', 'http://www.wikidata.org/entity/Q11424'), ('className', 'film')]
[('o', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows'), ('class', 'http://www.wikidata.org/entity/Q11424'), ('className', 'film')]
[('o', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('class', 'http://www.wikidata.org/entity/Q5398426'), ('className', 'television series')]
[('o', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes'), ('class', 'http://www.wikidata.org/entity/Q5398426'), ('className

Final query for this task

In [28]:
# write your final query
#Return the year of publication (take the first one if there are more publication dates) for each film and 
#television series related to the canon of Sherlock Holmes (the result set must be a list of triples film/tv 
#series IRI, label and year).
queryString = """
SELECT DISTINCT ?relatedWork ?name (year(MIN(?date)) AS ?firstYear)
WHERE {
   wd:Q2316684 wdt:P4969 ?relatedWork.
   ?relatedWork wdt:P577 | wdt:P580 ?date.
   ?relatedWork sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('relatedWork', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes'), ('firstYear', '1984')]
[('relatedWork', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes'), ('firstYear', '2013')]
[('relatedWork', 'http://www.wikidata.org/entity/Q1210893'), ('name', 'The Adventures of Sherlock Holmes and Dr. Watson'), ('firstYear', '1979')]
[('relatedWork', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('firstYear', '2012')]
[('relatedWork', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('firstYear', '2009')]
[('relatedWork', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('firstYear', '2010')]
[('relatedWork', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows'), ('firstYear', '2011')]
[('relatedWork', 'http://www.wikidata.org/entity/Q1764019'), ('name', 'Sherlock Holmes'), ('firstYear', '1939')]
8


## Task 5
Find who acted in more than one film or tv series related to the canon of Sherlock Holmes (the result set must be a list of triples actor/actress IRI, label and #film).

In [29]:
#RETURN PROPERTIES FOR SHERLOCK
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   wd:Q192837 ?p ?o .
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('name', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('name', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P170'

In [31]:
#RETURN SHERLOCK'S CAST MEMBERS
queryString = """
SELECT ?o ?name
WHERE {
   wd:Q192837 wdt:P161 ?o .
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q560286'), ('name', 'Mark Gatiss')]
[('o', 'http://www.wikidata.org/entity/Q472216'), ('name', 'Rupert Graves')]
[('o', 'http://www.wikidata.org/entity/Q309486'), ('name', 'Martin Freeman')]
[('o', 'http://www.wikidata.org/entity/Q1319582'), ('name', 'Una Stubbs')]
[('o', 'http://www.wikidata.org/entity/Q507322'), ('name', 'Andrew Scott')]
[('o', 'http://www.wikidata.org/entity/Q244674'), ('name', 'Benedict Cumberbatch')]
[('o', 'http://www.wikidata.org/entity/Q453740'), ('name', 'Amanda Abbington')]
[('o', 'http://www.wikidata.org/entity/Q2035905'), ('name', 'Zoe Telford')]
[('o', 'http://www.wikidata.org/entity/Q6272441'), ('name', 'Jonathan Aris')]
[('o', 'http://www.wikidata.org/entity/Q102893869'), ('name', 'Janice Acquah')]
[('o', 'http://www.wikidata.org/entity/Q1057334'), ('name', 'Louise Brealey')]
[('o', 'http://www.wikidata.org/entity/Q263691'), ('name', 'Lara Pulver')]
[('o', 'http://www.wikidata.org/entity/Q2947988'), ('name',

In [37]:
#return cast members acting in a film or tv series related to the canon of Sherlock Holmes
queryString = """
SELECT ?actor ?name 
WHERE {
   ?film wdt:P161 ?actor .
   wd:Q2316684 wdt:P4969 ?film.
   ?actor sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q22963868'), ('name', 'Vanessa Aspillaga')]
[('actor', 'http://www.wikidata.org/entity/Q391359'), ('name', 'David Harbour')]
[('actor', 'http://www.wikidata.org/entity/Q60493'), ('name', 'Lee Tergesen')]
[('actor', 'http://www.wikidata.org/entity/Q198684'), ('name', 'Wayne Duvall')]
[('actor', 'http://www.wikidata.org/entity/Q275658'), ('name', 'Wade Williams')]
[('actor', 'http://www.wikidata.org/entity/Q536686'), ('name', 'Sarah Danielle Madison')]
[('actor', 'http://www.wikidata.org/entity/Q60355'), ('name', 'Freda Foh Shen')]
[('actor', 'http://www.wikidata.org/entity/Q1102002'), ('name', 'Clive Russell')]
[('actor', 'http://www.wikidata.org/entity/Q1102002'), ('name', 'Clive Russell')]
[('actor', 'http://www.wikidata.org/entity/Q312399'), ('name', 'John Noble')]
[('actor', 'http://www.wikidata.org/entity/Q39979'), ('name', 'Chris Bauer')]
[('actor', 'http://www.wikidata.org/entity/Q603317'), ('name', 'Bill Irwin')]
[('actor', 'htt

In [38]:
#return also the number of film related to the canon of Sherlock Holmes for wich actors are part of the cast
queryString = """
SELECT ?actor ?name (COUNT(?film) AS ?numFilm)
WHERE {
   ?film wdt:P161 ?actor .
   wd:Q2316684 wdt:P4969 ?film.
   ?actor sc:name ?name.
}
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q39979'), ('name', 'Chris Bauer'), ('numFilm', '1')]
[('actor', 'http://www.wikidata.org/entity/Q447892'), ('name', 'Paul Sorvino'), ('numFilm', '1')]
[('actor', 'http://www.wikidata.org/entity/Q439886'), ('name', 'Ophelia Lovibond'), ('numFilm', '1')]
[('actor', 'http://www.wikidata.org/entity/Q526654'), ('name', 'Danielle Nicolet'), ('numFilm', '1')]
[('actor', 'http://www.wikidata.org/entity/Q728050'), ('name', 'Nigel Bruce'), ('numFilm', '1')]
[('actor', 'http://www.wikidata.org/entity/Q5336295'), ('name', 'Eddie McGee'), ('numFilm', '1')]
[('actor', 'http://www.wikidata.org/entity/Q271616'), ('name', 'Talia Balsam'), ('numFilm', '1')]
[('actor', 'http://www.wikidata.org/entity/Q575761'), ('name', "Michael O'Keefe"), ('numFilm', '1')]
[('actor', 'http://www.wikidata.org/entity/Q358990'), ('name', 'James Hong'), ('numFilm', '1')]
[('actor', 'http://www.wikidata.org/entity/Q943589'), ('name', 'Frankie Faison'), ('numFilm', '1')]
[('a

In [39]:
#return also the number of film related to the canon of Sherlock Holmes for wich actors are part of the cast
#order by descendant
queryString = """
SELECT ?actor ?name (COUNT(?film) AS ?numFilm)
WHERE {
   ?film wdt:P161 ?actor .
   wd:Q2316684 wdt:P4969 ?film.
   ?actor sc:name ?name.
}
ORDER BY DESC(?numFilm)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q160432'), ('name', 'Jude Law'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q45647'), ('name', 'Eddie Marsan'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q236167'), ('name', 'Kelly Reilly'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q1102002'), ('name', 'Clive Russell'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q165219'), ('name', 'Robert Downey Jr.'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q190386'), ('name', 'Rachel McAdams'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q440926'), ('name', 'Geraldine James'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q39979'), ('name', 'Chris Bauer'), ('numFilm', '1')]
[('actor', 'http://www.wikidata.org/entity/Q447892'), ('name', 'Paul Sorvino'), ('numFilm', '1')]
[('actor', 'http://www.wikidata.org/entity/Q439886'), ('name', 'Ophelia Lovibond'), ('numFilm', '1')]
[('

In [4]:
#FINAL QUERY TRY
queryString = """
SELECT ?actor ?name (COUNT(?film) AS ?numFilm)
WHERE {
   ?film wdt:P161 ?actor .
   wd:Q2316684 wdt:P4969 ?film.
   ?actor sc:name ?name.
}
GROUP BY ?actor ?name
HAVING(COUNT(?film) > 1)
ORDER BY DESC(?numFilm)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q236167'), ('name', 'Kelly Reilly'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q440926'), ('name', 'Geraldine James'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q190386'), ('name', 'Rachel McAdams'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q45647'), ('name', 'Eddie Marsan'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q1102002'), ('name', 'Clive Russell'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q160432'), ('name', 'Jude Law'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q165219'), ('name', 'Robert Downey Jr.'), ('numFilm', '2')]
7


Final query for this task

In [4]:
# write your final query
#Find who acted in more than one film or tv series related to the canon of Sherlock Holmes (the result set must be a list 
#of triples actor/actress IRI, label and #film).
queryString = """
SELECT ?actor ?name (COUNT(?film) AS ?numFilm)
WHERE {
   ?film wdt:P161 ?actor .
   wd:Q2316684 wdt:P4969 ?film.
   ?actor sc:name ?name.
}
GROUP BY ?actor ?name
HAVING(COUNT(?film) > 1)
ORDER BY DESC(?numFilm)
LIMIT 100

"""

print("Results")
x=run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q236167'), ('name', 'Kelly Reilly'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q440926'), ('name', 'Geraldine James'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q190386'), ('name', 'Rachel McAdams'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q45647'), ('name', 'Eddie Marsan'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q1102002'), ('name', 'Clive Russell'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q160432'), ('name', 'Jude Law'), ('numFilm', '2')]
[('actor', 'http://www.wikidata.org/entity/Q165219'), ('name', 'Robert Downey Jr.'), ('numFilm', '2')]
7


## Task 6
Identify the BGP for Primetime Emmy Award

In [2]:
#return properties for Benedict Cumberbatch
queryString = """
SELECT ?p ?name 
WHERE {
   wd:Q244674 ?p ?o.
   ?p sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('name', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P109'), ('name', 'signature')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), 

In [3]:
#return what Benedict Cumberbatch is nominated for
queryString = """
SELECT ?o ?name 
WHERE {
   wd:Q244674 wdt:P1411 ?o.
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1062026'), ('name', 'MTV Movie Award for Best Villain')]
[('o', 'http://www.wikidata.org/entity/Q628637'), ('name', "European Film Award – People's Choice Award for Best European Film")]
[('o', 'http://www.wikidata.org/entity/Q103916'), ('name', 'Academy Award for Best Actor')]
[('o', 'http://www.wikidata.org/entity/Q518675'), ('name', 'Screen Actors Guild Award for Outstanding Performance by a Cast in a Motion Picture')]
[('o', 'http://www.wikidata.org/entity/Q989453'), ('name', 'Primetime Emmy Award for Outstanding Lead Actor in a Miniseries or a Movie')]
[('o', 'http://www.wikidata.org/entity/Q2925700'), ('name', 'British Academy Television Award for Best Supporting Actor')]
[('o', 'http://www.wikidata.org/entity/Q2925701'), ('name', 'British Academy Television Award for Best Actor')]
[('o', 'http://www.wikidata.org/entity/Q6500801'), ('name', 'Laurence Olivier Award for Best Performance in a Supporting Role')]
[('o', 'http://www.wikid

In [4]:
#return what Primetime Emmy Award for Outstanding Lead Actor in a Miniseries or a Movie is instance of
queryString = """
SELECT ?o ?name 
WHERE {
   wd:Q989453 wdt:P31 ?o.
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1044427'), ('name', 'Primetime Emmy Award')]
[('o', 'http://www.wikidata.org/entity/Q38033430'), ('name', 'class of award')]
2


Final query for this task

In [5]:
# write your final query
queryString = """
SELECT ?o ?name 
WHERE {
   wd:Q989453 wdt:P31 ?o.
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1044427'), ('name', 'Primetime Emmy Award')]
[('o', 'http://www.wikidata.org/entity/Q38033430'), ('name', 'class of award')]
2


## Task 7
Return all the types of Primetime Emmy Award (the result set must be a list of couples award IRI and label).

In [10]:
#return all awards instances of Primetime Emmy Award
queryString = """
SELECT ?s ?name 
WHERE {
   ?s wdt:P31* wd:Q1044427.
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q18153303'), ('name', 'Primetime Emmy Award for Outstanding Unstructured Reality Program')]
[('s', 'http://www.wikidata.org/entity/Q28445729'), ('name', 'Primetime Emmy Award for Outstanding Actress in a Short Form Comedy or Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q989439'), ('name', 'Primetime Emmy Award for Outstanding Lead Actor in a Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q2604800'), ('name', 'Primetime Emmy Award for Outstanding Guest Actor in a Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q7243506'), ('name', 'Primetime Emmy Award for Outstanding Made for Television Movie')]
[('s', 'http://www.wikidata.org/entity/Q1277961'), ('name', 'Primetime Emmy Award for Outstanding Directing for a Comedy Series')]
[('s', 'http://www.wikidata.org/entity/Q3123491'), ('name', 'Primetime Emmy Award for Outstanding Writing for a Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q20714679'), ('name', 'Prim

In [11]:
#return all awards instances of Primetime Emmy Award with + operator
queryString = """
SELECT ?s ?name 
WHERE {
   ?s wdt:P31+ wd:Q1044427.
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q18153303'), ('name', 'Primetime Emmy Award for Outstanding Unstructured Reality Program')]
[('s', 'http://www.wikidata.org/entity/Q28445729'), ('name', 'Primetime Emmy Award for Outstanding Actress in a Short Form Comedy or Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q989439'), ('name', 'Primetime Emmy Award for Outstanding Lead Actor in a Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q2604800'), ('name', 'Primetime Emmy Award for Outstanding Guest Actor in a Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q7243506'), ('name', 'Primetime Emmy Award for Outstanding Made for Television Movie')]
[('s', 'http://www.wikidata.org/entity/Q1277961'), ('name', 'Primetime Emmy Award for Outstanding Directing for a Comedy Series')]
[('s', 'http://www.wikidata.org/entity/Q3123491'), ('name', 'Primetime Emmy Award for Outstanding Writing for a Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q20714679'), ('name', 'Prim

Final query for this task

In [12]:
# write your final query
#return all awards instances of Primetime Emmy Award with + operator
queryString = """
SELECT ?s ?name 
WHERE {
   ?s wdt:P31+ wd:Q1044427.
   ?s sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q18153303'), ('name', 'Primetime Emmy Award for Outstanding Unstructured Reality Program')]
[('s', 'http://www.wikidata.org/entity/Q28445729'), ('name', 'Primetime Emmy Award for Outstanding Actress in a Short Form Comedy or Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q989439'), ('name', 'Primetime Emmy Award for Outstanding Lead Actor in a Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q2604800'), ('name', 'Primetime Emmy Award for Outstanding Guest Actor in a Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q7243506'), ('name', 'Primetime Emmy Award for Outstanding Made for Television Movie')]
[('s', 'http://www.wikidata.org/entity/Q1277961'), ('name', 'Primetime Emmy Award for Outstanding Directing for a Comedy Series')]
[('s', 'http://www.wikidata.org/entity/Q3123491'), ('name', 'Primetime Emmy Award for Outstanding Writing for a Drama Series')]
[('s', 'http://www.wikidata.org/entity/Q20714679'), ('name', 'Prim

## Task 8
For each film and television series related to the canon of Sherlock Holmes, return the number of Primetime Emmy Awards they won (return also the objects who did not win an Award) (the result set must be a list of triples film/tv series IRI, label and #award).

In [4]:
#return awards won by sherlock tv series 
queryString = """
SELECT ?o ?name 
WHERE {
   wd:Q192837 wdt:P166 ?o.
   ?o sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q838121'), ('name', 'Peabody Awards')]
[('o', 'http://www.wikidata.org/entity/Q7243506'), ('name', 'Primetime Emmy Award for Outstanding Made for Television Movie')]
[('o', 'http://www.wikidata.org/entity/Q945887'), ('name', 'Primetime Emmy Award for Outstanding Supporting Actor in a Miniseries or a Movie')]
[('o', 'http://www.wikidata.org/entity/Q989453'), ('name', 'Primetime Emmy Award for Outstanding Lead Actor in a Miniseries or a Movie')]
[('o', 'http://www.wikidata.org/entity/Q17986500'), ('name', 'Primetime Emmy Award for Outstanding Cinematography for a Miniseries or Movie')]
[('o', 'http://www.wikidata.org/entity/Q2925700'), ('name', 'British Academy Television Award for Best Supporting Actor')]
[('o', 'http://www.wikidata.org/entity/Q7243513'), ('name', 'Primetime Emmy Award for Outstanding Special Visual Effects')]
[('o', 'http://www.wikidata.org/entity/Q7669617'), ('name', 'TCA Award for Outstanding Achievement in Movies, Minis

In [6]:
#return awards won by sherlock tv series which are instances of Primetime Emmy Awards
queryString = """
SELECT ?award ?name 
WHERE {
   wd:Q192837 wdt:P166 ?award.
   ?award wdt:P31 wd:Q1044427.
   ?award sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q7243506'), ('name', 'Primetime Emmy Award for Outstanding Made for Television Movie')]
[('award', 'http://www.wikidata.org/entity/Q945887'), ('name', 'Primetime Emmy Award for Outstanding Supporting Actor in a Miniseries or a Movie')]
[('award', 'http://www.wikidata.org/entity/Q989453'), ('name', 'Primetime Emmy Award for Outstanding Lead Actor in a Miniseries or a Movie')]
[('award', 'http://www.wikidata.org/entity/Q17986500'), ('name', 'Primetime Emmy Award for Outstanding Cinematography for a Miniseries or Movie')]
[('award', 'http://www.wikidata.org/entity/Q7243513'), ('name', 'Primetime Emmy Award for Outstanding Special Visual Effects')]
[('award', 'http://www.wikidata.org/entity/Q3403230'), ('name', 'Primetime Emmy Award for Outstanding Writing for a Miniseries, Movie or a Dramatic Special')]
[('award', 'http://www.wikidata.org/entity/Q17009259'), ('name', 'Primetime Emmy Award for Outstanding Music Composition for a Miniseries

In [9]:
#return for each work related to the canon of Sherlock Holmes the number of Primetime Emmy Award won
queryString = """
SELECT ?relatedWork ?name (COUNT(?award) AS ?totalEmmyAwards)
WHERE {
        wd:Q2316684 wdt:P4969 ?relatedWork.
        ?relatedWork wdt:P166 ?award.
        ?award wdt:P31 wd:Q1044427.
        ?relatedWork sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('relatedWork', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('totalEmmyAwards', '8')]
1


In [10]:
#return the number of Primetime Emmy Award won by Sherlock Holmes: A Game of Shadows
queryString = """
SELECT ?award ?name 
WHERE {
   wd:Q243643 wdt:P166 ?award.
   ?award wdt:P31 wd:Q1044427.
   ?award sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [12]:
#add optional to return also who won 0 awards
queryString = """
SELECT ?relatedWork ?name (COUNT(?award) AS ?totalEmmyAwards)
WHERE {
        wd:Q2316684 wdt:P4969 ?relatedWork.
        OPTIONAL {?relatedWork wdt:P166 ?award.}
        ?award wdt:P31 wd:Q1044427.
        ?relatedWork sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('relatedWork', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('totalEmmyAwards', '8')]
1


In [13]:
#add optional to return also who won 0 awards (wrong query)
queryString = """
SELECT ?relatedWork ?name (COUNT(?award) AS ?totalEmmyAwards)
WHERE {
        wd:Q2316684 wdt:P4969 ?relatedWork.
        OPTIONAL {?relatedWork wdt:P166 ?award.}
        OPTIONAL {?award wdt:P31 wd:Q1044427.}
        ?relatedWork sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('relatedWork', 'http://www.wikidata.org/entity/Q1210893'), ('name', 'The Adventures of Sherlock Holmes and Dr. Watson'), ('totalEmmyAwards', '84')]
[('relatedWork', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('totalEmmyAwards', '84')]
[('relatedWork', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes'), ('totalEmmyAwards', '84')]
[('relatedWork', 'http://www.wikidata.org/entity/Q1764019'), ('name', 'Sherlock Holmes'), ('totalEmmyAwards', '84')]
[('relatedWork', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('totalEmmyAwards', '16')]
[('relatedWork', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('totalEmmyAwards', '1')]
[('relatedWork', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes'), ('totalEmmyAwards', '84')]
[('relatedWork', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows'), ('totalEmmyAwards', '84')]
8


In [14]:
#add optional to return also who won 0 awards
queryString = """
SELECT ?relatedWork ?name (COUNT(?award) AS ?totalEmmyAwards)
WHERE {
        wd:Q2316684 wdt:P4969 ?relatedWork.
        OPTIONAL {?relatedWork wdt:P166 ?award.
                  ?award wdt:P31 wd:Q1044427.}
        ?relatedWork sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('relatedWork', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('totalEmmyAwards', '0')]
[('relatedWork', 'http://www.wikidata.org/entity/Q1210893'), ('name', 'The Adventures of Sherlock Holmes and Dr. Watson'), ('totalEmmyAwards', '0')]
[('relatedWork', 'http://www.wikidata.org/entity/Q1764019'), ('name', 'Sherlock Holmes'), ('totalEmmyAwards', '0')]
[('relatedWork', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes'), ('totalEmmyAwards', '0')]
[('relatedWork', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('totalEmmyAwards', '8')]
[('relatedWork', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('totalEmmyAwards', '0')]
[('relatedWork', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes'), ('totalEmmyAwards', '0')]
[('relatedWork', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows'), ('totalEmmyAwards', '0')]
8


Final query for this task

In [13]:
# write your final query
queryString = """
SELECT ?relatedWork ?name (COUNT(?award) AS ?totalEmmyAwards)
WHERE {
        wd:Q2316684 wdt:P4969 ?relatedWork.
        OPTIONAL {?relatedWork wdt:P166 ?award.
                  ?award wdt:P31 wd:Q1044427.}
        ?relatedWork sc:name ?name.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('relatedWork', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes'), ('totalEmmyAwards', '0')]
[('relatedWork', 'http://www.wikidata.org/entity/Q1210893'), ('name', 'The Adventures of Sherlock Holmes and Dr. Watson'), ('totalEmmyAwards', '0')]
[('relatedWork', 'http://www.wikidata.org/entity/Q1764019'), ('name', 'Sherlock Holmes'), ('totalEmmyAwards', '0')]
[('relatedWork', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes'), ('totalEmmyAwards', '0')]
[('relatedWork', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock'), ('totalEmmyAwards', '8')]
[('relatedWork', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary'), ('totalEmmyAwards', '0')]
[('relatedWork', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes'), ('totalEmmyAwards', '0')]
[('relatedWork', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows'), ('totalEmmyAwards', '0')]
8
